# Pytorch dataset creation

> pytorch dataset will be created here

In [ ]:
#| default_exp dataloader_creation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from PIL import Image
from typing import List, Callable, Tuple, Dict, Union
from pathlib import Path
from fastcore.all import *
import cv2
import torch
from fastcore.all import *
from fastcore.foundation import *

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#| export
def get_transforms(*, data):
    if data == 'train':
        return A.Compose([
            #A.Resize(256, 256),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.Transpose(p=0.5),
            A.ShiftScaleRotate(shift_limit=0.01, scale_limit=0.04, rotate_limit=0, p=0.25),
            #A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            A.Normalize(mean=[0, 0, 0], std=[1/255, 1/255, 1/255]),
            ToTensorV2(),
        ])

    elif data == 'valid':
        return A.Compose([
            #A.Resize(256, 256),
            A.Normalize(mean=[0, 0, 0], std=[1/255, 1/255, 1/255]),
            #A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2(),
        ])

In [ ]:
from fastcore.all import *
from cv_tools.core import *

In [ ]:
#| export
class SegmentationDataset(Dataset):
    def __init__(
                self, 
                image_path:Union[Path, str],
                mask_path:Union[Path, str],
                transform=None,
                exts='.png'
                ):
        # getting images and masks
        self.image_path = Path(image_path)
        self.mask_path = Path(mask_path)
        self.transform = transform
        self.images = Path(self.image_path).ls(file_exts=exts)
        self.masks = Path(self.mask_path).ls(file_exts=exts)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_fn = self.images[idx]
        # Assuming the mask file name is same as image file name
        name_ = Path(image_fn).name
        image = cv2.imread(f"{image_fn}", cv2.IMREAD_GRAYSCALE)
        mask_fn = Path(self.mask_path, name_)

        mask = cv2.imread(f"{mask_fn}",cv2.IMREAD_GRAYSCALE)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [ ]:
Path.cwd()

Path('n:/homes/hasan/easy_front/easy-front-pin-detection/nbs')

In [ ]:

root_path= Path(r'm:/data/projects/easy_front_detection/eberhard_data')
im_path = f'{root_path}/images'
mask_path = f'{root_path}/masks'

In [ ]:
TRAIN_SPLIT=0.8

In [ ]:
train_len = int(TRAIN_SPLIT *len(dataset))
val_len = len(dataset) - train_len
train_len, val_len

(271, 68)

In [ ]:
train_dataset, val_dataset = random_split(
    dataset,
    lengths=[train_len, val_len]
)

In [ ]:
len(train_dataset), len(val_dataset)

(271, 68)

In [ ]:
#| export
def repeat_collate_fn(batch, batch_size=4):
    images, masks = zip(*batch)

    images = list(images)
    masks = list(masks)

    num_to_add = batch_size - len(images)
    if num_to_add > 0:
        for i in range(num_to_add):
            index = i%len(images)
            images.append(images[index])
            masks.append(masks[index])
    return torch.stack(images), torch.stack(masks)


In [ ]:
#| export
def create_pytorch_dataloader(
    split_type:str, # in case of 'random' randomly data will be splitted
    split_per:float, # percentage of training data
    batch_size:int,
    image_path:Union[Path, str],
    mask_path:Union[Path, str],
    trn_transforms:Callable, # a callable function trn_transforms 
    val_transforms:Callable, # a callable function val_transforms
    exts:str='.png',  # image  and mask data extensions
    collate_fn:Callable=repeat_collate_fn,
    num_workers:str=4
   ):

    'Create pytorch dataloader based on the argument'

    full_dataset = SegmentationDataset(
                                image_path=image_path,
                                mask_path=mask_path,
                                exts=exts
                              )

    print(f' Number of images found = {len(full_dataset)}')
    if split_type == 'random':
        train_len = int(split_per * len(full_dataset))
        val_len = len(full_dataset) - train_len


        indices = torch.randperm(len(full_dataset)).tolist()
        train_indices, val_indices = indices[:train_len], indices[train_len:]

        train_ds = Subset(full_dataset, train_indices)
        val_ds = Subset(full_dataset, val_indices)


        #train_ds, val_ds = random_split(dataset, lengths=[train_len, val_len])
        print(f' training dataset length = {len(train_ds)} and validation dataset length=  {len(val_ds)}')
        

        train_ds.dataset.transform = trn_transforms
        val_ds.dataset.transform = val_transforms

        train_dl = DataLoader(
                              train_ds, 
                              batch_size=batch_size,
                              shuffle=True, 
                              num_workers=num_workers, 
                              pin_memory=True,
                              collate_fn=collate_fn
        )
        val_dl = DataLoader(
                              val_ds, 
                              batch_size=batch_size,
                              shuffle=False, 
                              num_workers=num_workers, 
                              pin_memory=True,
                              collate_fn=collate_fn
        )
        return train_dl, val_dl

    else:
        raise NotImplementedError("Only random split is implemented")
    




In [ ]:
Path(im_path).ls(file_exts='.png')

(#339) [Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_0_A.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_0_B.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_10_A.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_10_B.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_11_A.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_11_B.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_12_A.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_12_B.png'),Path('m:/data/projects/easy_front_detection/eberhard_data/images/0_0_CroppedImg_20231018_92042317_13_A.png'),Path('m:/data

In [ ]:
train_dl, val_dl = create_pytorch_dataloader(
    split_type='random',
    split_per=0.8,
    batch_size=2,
    image_path=im_path,
    mask_path=mask_path,
    transforms=None,
    num_workers=0,
    collate_fn=repeat_collate_fn

)

 Number of images found = 339
 training dataset length = 271 and validation dataset length=  68


In [ ]:
#| export
def visualize_batch(images, masks, num_images=4):
    fig, axs = plt.subplots(1,num_images, figsize=(5, num_images*5))
    for idx, (image, mask) in enumerate(zip(images, masks)):
        if idx >= num_images:
            break
        axs[idx].imshow(image.permute(1, 2, 0)[:,:,0], cmap='gray')
        axs[idx].imshow(mask.squeeze(), cmap='jet', alpha=0.3)  # overlay mask on image
        axs[idx].axis('off')
        axs[idx].set_title('Image with Mask')
    plt.tight_layout()
    plt.show()

In [ ]:

images, masks = next(iter(train_dl))
#visualize_batch(images=images, masks=masks, num_images=2)

NameError: name 'train_dl' is not defined

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('09_dataset_creation.ipynb')